In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
#Load Enviroment variables in a file called .env
load_dotenv()
api_key = os.getenv('OPEN_API_KEY')
#Check the key
if not api_key:
    print("No Api key was found")
elif api_key.strip() !=api_key:
    print("An API key was foun,but it looks like it might have a space or tab charachters at the start or end-please remove it")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
import openai
openai.api_key=api_key

In [4]:
import requests
from bs4 import BeautifulSoup
class Website: 
    """ utility class to represent a website that we have scraped """
    def __init__(self,url):
        """ create this website object from the given URL using the Beautifull Library"""
        self.url = url
        response = requests.get(url)
        if response.status_code!=200:
            raise Exception(f"Failed to fetch URL :{url},Status Code :{response.status_code}")
        soup = BeautifulSoup(response.content,'html.parser')
        self.title = soup.title.string if soup.title else "No Title Found "
        # Remove unnecessary elements 
        for irrelevent in soup.body(["script","style","img","input"]):
            irrelevent.decompose()
        self.text = soup.body.get_text(separator ="\n",strip=True) if soup.body else "No text founde"
            

In [39]:
#web = Website("http://bogariagency.com/")
#print(web.title)
#print(web.text)

In [40]:
#Define our system prompt 
system_prompt = "You are an assistant that analyzes the contents of a website\
and provides a short summary,ignoring text that might be navigation related \
Respond in markdown"

In [73]:
def user_prompt_for(website):
    user_prompt = f"You are looking at the website titled: {website.title}\n\n"  # ✅ Corrected
    user_prompt += "The contents of this website are as follows:\n"
    user_prompt += "Please provide a short summary of this website in markdown.\n"
    user_prompt += "If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text  # ✅ Correct usage of website.text
    return user_prompt



In [63]:
system_prompt

'You are an assistant that analyzes the contents of a websiteand provides a short summary,ignoring text that might be navigation related Respond in markdown'

In [64]:
#print(user_prompt_for(web))

In [74]:
def messages_for(website):
    return [
        {"role": "system", "content": "You are an AI assistant that summarizes websites."},
        {"role": "user", "content": user_prompt_for(website)}  # ✅ Pass website object, not website.text
    ]

In [75]:
#messages_for(web)

In [76]:
def summarize(url):
    website=Website(url)
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [77]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website serves as a personal platform for sharing insights and resources related to large language models (LLMs), coding, and AI. Ed, the co-founder and CTO of Nebula.io, is passionate about leveraging AI for talent discovery and engagement. The site includes an arena called "Outsmart," where LLMs compete in diplomacy and strategy. \n\n## About Ed\n- Background in writing code and experimenting with LLMs.\n- Interest in DJing and electronic music production.\n- Previous experience as the founder of AI startup untapt, acquired in 2021.\n\n## News and Announcements\n- **January 23, 2025**: Released resources for the "LLM Workshop – Hands-on with Agents."\n- **December 21, 2024**: Welcomed "SuperDataScientists" to the community.\n- **November 13, 2024**: Provided resources for "Mastering AI and LLM Engineering."\n- **October 16, 2024**: Shared resources on "From Software Engineer to AI Data Scientist."\n\nFor more information or t

In [84]:
def display_summary(url):
    summary=summarize(url)
    display(Markdown(summary))

In [89]:
display_summary("https://www.tgju.org/currency")

# Summary of نرخ ارز | شبکه اطلاع‌رسانی طلا و ارز

The "نرخ ارز" website provides comprehensive information about the gold and currency markets in Iran. It includes real-time data on gold prices (such as 18-carat and 24-carat gold), various currency rates (including USD, EUR, GBP, AED), and local exchange rates. 

## Key Features:
- **Gold and Currency Pricing**: 
  - Live updates on gold prices and comparisons (مثقال, آبشده).
  - Exchange rates for major currencies.
- **Market Analysis**: 
  - Information on the gold, currency, and stock markets.
  - Historical price archives for various currencies and commodities.
- **Mobile Application**: 
  - Availability of apps for Android and iPhone for easy access to market data.
  
## Recent News / Announcements:
The website also features recent price changes in currencies and gold:
- **USD Rate**: 902,850 IRR
- **EUR Rate**: 940,200 IRR
- **GBP Rate**: 1,127,400 IRR
- **Gold Rate**: 64,171,000 IRR for 18-carat gold.
  
## Additional Information:
- The site explains different types of exchange rates including free market rates, state rates, SENA rates, NEEMA rates, and more. It discusses their significance and the way they impact transactions in Iran.

This resource serves as an essential tool for individuals needing up-to-date financial information regarding gold and the currency exchange market in Iran.